In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# 모델 및 토크나이저 로드
model_path = "./model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# 요약할 텍스트
text = """
뇌-컴퓨터 인터페이스(Brain-Computer Interface, BCI) 기술이 급격히 발전하면서 신경 장애를 가진 환자들에게 새로운 삶의 가능성을 열고 있다.
단순히 운동 기능을 복원하는 수준을 넘어서 일상 생활이나 비디오 게임과 같은 여가 활동을 가능하게 하는 데까지 도달했다.
미국 미시간대 연구진은 심각한 운동 장애를 겪는 환자들이 비디오 게임과 같은 여가 활동을 할 수 있도록 돕는 BCI 기술을 개발했다고 밝혔다.
연구 결과는 국제 학술지 ‘네이처 메디신(Nature Medicine)’에 21일 공개됐다.
BCI 기술은 뇌와 컴퓨터를 연결해 신체 기능을 복원하거나, 인간의 의도를 외부 기기로 전달하는 기술이다.
기존 BCI 기술은 마비 환자의 운동 기능 일부를 복원하는 데 사용돼 왔으나, 손가락 움직임과 같은 정교한 동작을 재현하는 데는 한계가 있었다.
따라서 타이핑이나 악기 연주, 게임 컨트롤러 조작과 같은 활동을 구현하기에는 어려웠다.
연구진은 척수 손상으로 사지마비를 앓는 환자의 뇌에 전극을 이식해 손 움직임과 관련된 신경 신호를 기록했다.
환자가 가상 손의 움직임을 관찰할 때 뇌에서 나오는 신경 데이터를 머신러닝 알고리즘으로 분석해 특정 손가락 움직임과 연관된 신호를 식별했다.
이를 기반으로 뇌에 전극을 이식한 환자가 손가락을 움직이는 것을 상상하는 것만으로 비디오 게임 속 가상 드론을 조종할 수 있게 했다.
환자는 가상 드론의 속도와 방향을 자유롭게 조절하며 여러 장애물을 통과하는 비디오 게임을 성공적으로 수행했다.
"""

# 요약 지시를 포함한 입력 생성
instruction = "위 텍스트를 간결하게 요약해 주세요."
messages = [
    {"role": "user", "content": f"{instruction}\n\n{text}"}
]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [4]:
# 텍스트 생성
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:
# 생성된 텍스트 디코딩 및 출력
summary = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print("요약 결과:")
print(summary)

요약 결과:
뇌-컴퓨터 인터페이스(Brain-Computer Interface, BCI) 기술은 신경 장애를 가진 환자들에게 새로운 삶의 가능성을 제공하고 있습니다. 연구진은 심각한 운동 장애를 겪는 환자들이 비디오 게임과 같은 여가 활동을 할 수 있도록 BCI 기술을 개발했습니다. 이 기술은 뇌와 컴퓨터를 연결하여 신체 기능을 복원하거나, 인간의 의도를 외부 기기로 전달하는 역할을 합니다. 이전에는 마비 환자의 운동 기능을 일부 복원하는 데 사용되었습니다. 그러나 연구진은 척수 손상으로 사지마비를 앓는 환자의 뇌에 전극을 이식하고, 뇌에서 나오는 신경 데이터를 분석하여 손가락 움직임을 재현하는 신호를 식별했습니다. 이를 기반으로 환자는 가상 손의 움직임을 상상하여 비디오 게임 속 가상 드론을 조종할 수 있게 되었습니다.assistant

뇌-컴퓨터 인터페이스(Brain-Computer Interface, BCI) 기술은 신경 장애를 가진 환자들에게 새로운 삶의 가능


In [8]:
# 주관식 문제 생성 지시
instruction = "다음 요약 내용을 기반으로 문제와 정답을 생성해주세요.\n\n"

messages = [
    {"role": "user", "content": f"{instruction}{summary}"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# 텍스트 생성
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [7]:
# 생성된 문제와 정답 출력
qa_pair = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print("생성된 주관식 문제와 정답:")
print(qa_pair)

생성된 주관식 문제와 정답:
뇌-컴퓨터 인터페이스(Brain-Computer Interface, BCI) 기술은 신경 장애를 가진 환자들에게 새로운 삶의 가능성을 제공하고 있습니다. 연구진은 심각한 운동 장애를 겪는 환자들이 비디오 게임과 같은 여가 활동을 할 수 있도록 BCI 기술을 개발했습니다. 이 기술은 뇌와 컴퓨터를 연결하여 신체 기능을 복원하거나, 인간의 의도를 외부 기기로 전달하는 역할을 합니다. 이전에는 마비 환자의 운동 기능을 일부 복원하는 데 사용되었습니다. 그러나 연구진은 척수 손상으로 사지마비를 앓는 환자의 뇌에 전극을 이식하고, 뇌에서 나오는 신경 데이터를 분석하여 손가락 움직임을 재현하는 신호를 식별했습니다. 이를 기반으로 환자는 가상 손의 움직임을 상상하여 비디오 게임 속 가상 드론을 조종할 수 있게 되었습니다.assistant

뇌-컴퓨터 인터페이스(Brain-Computer Interface, BCI) 기술은 신경 장애를 가진 환자들에게 새로운 삶의 가능
